In [1]:
import calendar
import datetime
import logging
import pprint

import ee
from IPython.display import Image

import openet.ssebop as ssebop
import utils

ee.Initialize()

logging.basicConfig(level=logging.DEBUG, format='%(message)s')

#### Inputs

In [2]:
start_date = '2017-07-01'
end_date = '2017-07-31'
overwrite_flag = False

#### Tcorr Input Feature Collection

In [3]:
tcorr_ftr_coll_id = 'projects/usgs-ssebop/tcorr/topowx_median_v0_scene'

#### Tcorr Output Image Collection

In [4]:
tcorr_img_name = 'topowx_median_v0'
tcorr_img_coll_id = 'projects/usgs-ssebop/tcorr_image/{}'.format(tcorr_img_name)
tcorr_img_coll = ee.ImageCollection(tcorr_img_coll_id);

#### Study Area

In [5]:
export_geom = ee.Geometry.Rectangle(-125, 25, -65, 50)
# export_geom = ee.Geometry.Rectangle(-124, 35, -119, 42)
export_crs = 'EPSG:4326'
export_region = export_geom.bounds(1, export_crs).coordinates().getInfo()[0][:-1]
print(export_region)

[[-125.0, 25.0], [-65.0, 25.0], [-65.0, 53.99478518121347], [-125.0, 53.99478518121347]]


#### Tmax Collection

In [6]:
tmax_name = 'topowx_median_v0'
tmax_version = 'median_v0'
tmax_coll_id = 'projects/usgs-ssebop/tmax/{}'.format(tmax_name)

tmax_coll = ee.ImageCollection(tmax_coll_id)
tmax_img = ee.Image(tmax_coll.first()).set('TMAX_VERSION', tmax_version)
# tmax_image = ee.Image(tmax_coll.filter(doy_filter).first())\
#     .set('TMAX_VERSION', tmax_version)

# Hardcode properties for now (but they could be retrieved dynamically from tmax_coll)
tmax_geo = [0.00833333333333333, 0.0, -125.00416722008521, 0.0, -0.00833333333333333, 51.19583312184854]
# tmax_geo = [0.00833333329998709, 0.0, -125.00416722008521, 0.0, -0.00833333329998709, 51.19583312184854]
tmax_crs = 'EPSG:4326'
tmax_shape = [7000, 3250]
tmax_extent = [tmax_geo[2], tmax_geo[5] + tmax_shape[1] * tmax_geo[4], 
               tmax_geo[2] + tmax_shape[0] * tmax_geo[0], tmax_geo[5]]
# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['transform'])
# print(ee.Image(tmax_median_coll.first()).projection().getInfo()['crs'])
# print(ee.Image(tmax_median_coll.first()).getInfo()['bands'][0]['dimensions'])

Image(url=tmax_img.getThumbURL({'min': 270, 'max': 330, 'region': export_region}))
# embed=True, format='png'

#### Export Extent, Shape, Geo

In [7]:
# Adjust the study area extent to match the Tmax Image Collection transform

# export_cs = 0.008333333333333333333333  # ~800m
# export_cs = 0.016666666666666666666666  # ~1600m
export_cs = 0.033333333333333333333333  # ~3200m
export_crs = 'EPSG:4326'

# Compute clipped Tmax grid (this is a disaster of code)
export_xy = ee.Array(export_geom.bounds(1, export_crs).coordinates().get(0)).transpose().toList();
export_xmin = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.min()));
export_ymin = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.min()));
export_xmax = ee.Number(ee.List(export_xy.get(0)).reduce(ee.Reducer.max()));
export_ymax = ee.Number(ee.List(export_xy.get(1)).reduce(ee.Reducer.max()));
# Snap to Tmax grid
export_xmin = export_xmin.subtract(tmax_extent[0]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[0]);
export_ymin = export_ymin.subtract(tmax_extent[3]).divide(export_cs).floor().multiply(export_cs).add(tmax_extent[3]);
export_xmax = export_xmax.subtract(tmax_extent[0]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[0]);
export_ymax = export_ymax.subtract(tmax_extent[3]).divide(export_cs).ceil().multiply(export_cs).add(tmax_extent[3]);
#  Limit to Tmax grid
export_xmin = export_xmin.max(tmax_extent[0]).min(tmax_extent[2]);
export_ymin = export_ymin.max(tmax_extent[1]).min(tmax_extent[3]);
export_xmax = export_xmax.min(tmax_extent[0]).max(tmax_extent[2]);
export_ymax = export_ymax.min(tmax_extent[1]).max(tmax_extent[3]);

export_extent = ee.List([export_xmin, export_ymin, export_xmax, export_ymax]);
export_geo = ee.List([export_cs, 0.0, export_xmin, 0.0, -export_cs, export_ymax]).getInfo();
export_shape = ee.List([
  export_xmax.subtract(export_xmin).abs().divide(export_cs).int(),
  export_ymax.subtract(export_ymin).abs().divide(export_cs).int()]).getInfo();
print(export_shape);
print(export_geo);

[1749, 786]
[0.03333333333333333, 0.0, -125.00416722008521, 0.0, -0.03333333333333333, 51.19583312184854]


#### Get Current Task and Asset Lists

In [8]:
tasks = utils.get_ee_tasks()
if logging.getLogger().getEffectiveLevel() == logging.DEBUG:
    logging.debug('  Tasks: {}'.format(len(tasks)))


Active Tasks
  None
  Tasks: 0


In [9]:
asset_list = utils.get_ee_assets(tcorr_img_coll_id, shell_flag=True)
logging.debug('Displaying first 10 images in collection')
logging.debug(asset_list[:10])

Displaying first 10 images in collection
['projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170716', 'projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170717', 'projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170718']


#### Export the Tcorr Image for each date

In [10]:
start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

for export_dt in utils.date_range(start_dt, end_dt, days=1, skip_leap_days=False):   
    logging.info('\n{}'.format(export_dt.strftime('%Y-%m-%d')))
    
    task_id = 'tcorr_image_{}_{}'.format(tcorr_img_name, export_dt.strftime('%Y%m%d'))
    asset_id = '{}/{}'.format(tcorr_img_coll_id, export_dt.strftime('%Y%m%d'))
    logging.debug('  Task ID: {}'.format(task_id))
    logging.debug('  Asset ID: {}'.format(asset_id))
    
    if overwrite_flag:
        if export_id in tasks.keys():
            logging.debug('  Task already submitted, cancelling')
            ee.data.cancelTask(tasks[task_id])
        # This is intentionally not an "elif" so that a task can be
        # cancelled and an existing image/file/asset can be removed
        if export_path in asset_list:
            logging.debug('  Asset already exists, removing')
            ee.data.deleteAsset(asset_id)
            # subprocess.call(['earthengine', 'rm', export_path])
    else:
        if task_id in tasks.keys():
            logging.debug('  Task already submitted, skipping')
            continue
        elif asset_id in asset_list:
            logging.debug('  Asset already exists, skipping')
            continue

    # Build and merge the Landsat collections
    l8_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA')\
        .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
        .filterBounds(tmax_img.geometry())\
        .filterBounds(export_geom)
    l7_coll = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA')\
        .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
        .filterBounds(tmax_img.geometry())\
        .filterBounds(export_geom)
    landsat_coll = l8_coll.merge(l7_coll)
    # l5_coll = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
    #     .filterDate(export_dt, export_dt + datetime.timedelta(days=1))\
    #     .filterBounds(tmax_image.geometry())\
    #     .filterBounds(export_geom)
    # landsat_coll = l8_coll.merge(l7_coll).merge(l5_coll)

    # Join the Tcorr feature collection to the Landsat image collection
    join_coll = ee.Join.saveFirst(matchKey='tcorr') \
        .apply(landsat_coll, 
               ee.FeatureCollection(tcorr_ftr_coll_id), 
               ee.Filter.stringEndsWith(leftField='system:index', 
                                        rightField='SCENE_ID'))

    # Build the Tcorr image from Tcorr features
    def tcorr_image_func(landsat_img):
        tcorr_ftr = ee.Feature(landsat_img.get('tcorr'))
        return tmax_img.select([0], ['tcorr'])\
            .clip(landsat_img.geometry())\
            .multiply(0).add(ee.Number(tcorr_ftr.get('TCORR')))\
            .copyProperties(landsat_img, ['system:time_start', 'SCENE_ID'])
    tcorr_coll = ee.ImageCollection(join_coll.map(tcorr_image_func))
    tcorr_img = tcorr_coll.mean()\
        .setMulti({
            'system:time_start': utils.millis(export_dt),
            'SCENE_ID_LIST':  ee.List(ee.Dictionary(tcorr_coll.aggregate_histogram('system:index')).keys()),
            'SSEBOP_VER': ssebop.__version__,
            'TMAX_SOURCE': tmax_name.upper(),
            'TMAX_VERSION': tmax_version,
            'EXPORTDATE': datetime.datetime.today().strftime('%Y-%m-%d'),
        })

    # Image(url=ee.Image(tcorr_img).getThumbURL({
    #     'min': 0.975, 'max': 0.995, 'region': export_region,
    #     'palette': ['#EFE7E1', '#003300']}))
    # # embed=True, format='png'

    task = ee.batch.Export.image.toAsset(
        image=ee.Image(tcorr_img),
        description=task_id,
        assetId=asset_id,
        crs=export_crs,
        crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
        dimensions='{0}x{1}'.format(*export_shape),
        # scale=0.04,
    )
    task.start()


2017-07-01
  Task ID: tcorr_image_topowx_median_v0_20170701
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170701

2017-07-02
  Task ID: tcorr_image_topowx_median_v0_20170702
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170702

2017-07-03
  Task ID: tcorr_image_topowx_median_v0_20170703
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170703

2017-07-04
  Task ID: tcorr_image_topowx_median_v0_20170704
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170704

2017-07-05
  Task ID: tcorr_image_topowx_median_v0_20170705
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170705

2017-07-06
  Task ID: tcorr_image_topowx_median_v0_20170706
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170706

2017-07-07
  Task ID: tcorr_image_topowx_median_v0_20170707
  Asset ID: projects/usgs-ssebop/tcorr_image/topowx_median_v0/20170707

2017-07-08
  Task ID: tcorr_image_topowx_median_v0_20170708
  Asset ID: pro